In [1]:
import pandas as pd
import os
import glob
import numpy as np
import strsimpy
print('setup done!')
import hashlib
from tqdm import tqdm


def load_data(path):
    #path = os.getcwd()
    path = os.path.join(path, '*.csv')
    print(path)
    csv_files = glob.glob(path)
    print(len(csv_files))
    df = pd.read_csv(csv_files[0])
    for i, file in enumerate(csv_files):
        if i == 0:
            continue
        tmp = pd.read_csv(file)
        df = pd.concat([df, tmp], axis=0)
    return df


setup done!


In [2]:
levenshtein  = strsimpy.NormalizedLevenshtein()

def levenshtein_score(x,y):
    if (x=='unknown' or y =='unknown'):
        return 1
    return 1 - levenshtein.distance(x,y)

def difference_score(x,y):
    if (x=='unknown' or y =='unknown'):
        return 1
    return 1 if x==y else 0

def clust_filter(arr):
    flag = [False]*len(arr)
    cluster = []
    for i in tqdm(range(0,len(arr)-1)):
        if flag[i]==False:
            list_id = []
            list_id.append(arr[i][10])
            for j in range(i + 1, len(arr)):
                if flag[j]==False:
                    name = difference_score(arr[i][0],arr[j][0])
                    ram = difference_score(arr[i][2],arr[j][2])
                    rom = difference_score(arr[i][3],arr[j][3])
                    color = levenshtein_score(arr[i][6],arr[j][6])
                    status = difference_score(arr[i][7], arr[j][7])
                    score = 0.5 * name + 0.05 * ram + 0.15 * rom + 0.2 * color + 0.1 * status
                    if score >= 0.9:
                        flag[j] = True
                        list_id.append(arr[j][10])
            list_id = list(set(list_id))
            arr[i].append(list_id)
            cluster.append(arr[i])
            
    return cluster
def encrypt_string(hash_string):
    sha_signature = hashlib.sha256(hash_string.encode()).digest()
    return "".join(["{:x}".format(b) for b in sha_signature])

In [3]:
path = '../data/production'
df = load_data(path)

../data/production/*.csv
10


In [4]:
df = df.drop(['Unnamed: 0'], axis=1)

In [5]:
df.drop_duplicates(inplace=True)
len(df)

2611

In [6]:
df.head()

,name,url,ram,rom,price,createdDate,color,status,url_img,store
0,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-moi/ipho...,6,128.0,29790000,2023-01-10 20:58:07.383649,PURPLE,0,unknown,24hstore
1,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-moi/ipho...,6,256.0,33290000,2023-01-10 20:58:07.383649,PURPLE,0,unknown,24hstore
2,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-moi/ipho...,6,512.0,39490000,2023-01-10 20:58:07.383649,PURPLE,0,unknown,24hstore
3,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-moi/ipho...,6,1.0,44990000,2023-01-10 20:58:07.383649,PURPLE,0,unknown,24hstore
4,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-moi/ipho...,6,128.0,29790000,2023-01-10 20:58:07.383649,YELLOW,0,unknown,24hstore


In [7]:
df.columns

Index(['name', 'url', 'ram', 'rom', 'price', 'createdDate', 'color', 'status',
       'url_img', 'store'],
      dtype='object')

In [8]:
df['id'] = df['name'].astype(str) + df['url'].astype(str) + df['rom'].astype(str) + df['color'] + \
 df['status'].astype(str) + df['store'].astype(str)
df['id'] = df['id'].apply(encrypt_string)

In [10]:
pd.set_option('display.max_colwidth', None)

In [16]:
df_tmp = df.drop_duplicates(subset=['id'])

In [18]:
df_tmp.reset_index(drop=True, inplace=True)
df_tmp.to_json('./products.json', 'records')

In [19]:
arr = df.values.tolist()
arr[0]

['IPHONE 14 PRO MAX',
 'https://24hstore.vn/iphone-14-pro-max-moi/iphone-14-pro-max-1tb-p6626',
 6,
 128.0,
 29790000,
 '2023-01-10 20:58:07.383649',
 'PURPLE',
 0,
 'unknown',
 '24hstore',
 '7760cef25f6a9dbe92772fba4897cada8e20c489abaddd7d73f6664d23b892c']

In [20]:
cluster = clust_filter(arr)

100%|██████████| 2610/2610 [00:12<00:00, 214.60it/s]


In [22]:
cluster[0]

['IPHONE 14 PRO MAX',
 'https://24hstore.vn/iphone-14-pro-max-moi/iphone-14-pro-max-1tb-p6626',
 6,
 128.0,
 29790000,
 '2023-01-10 20:58:07.383649',
 'PURPLE',
 0,
 'unknown',
 '24hstore',
 '7760cef25f6a9dbe92772fba4897cada8e20c489abaddd7d73f6664d23b892c',
 ['022a62cc2d9545454dc98e0828444422c806b689cc71ed449381d85049a',
  '93778de7653ac05e8cbd91919466c8d1a352619a25aa14519c3b3c4efff74',
  '7760cef25f6a9dbe92772fba4897cada8e20c489abaddd7d73f6664d23b892c',
  'b531d9104337687680e259d0b62a425f457e0df70535688779b284cbe57e2a',
  'fc456ea1881fc1c56203096aacdcc841f6236b8ca32652f9ecb9fa3ce3afa4',
  '6f25f4c965543de890b2e8d2c073f54cb33c8fd97406e6e985441a06dd3a96a',
  '31228e4c0f752a31e9faa9cfafa1b8ad73e5f85c9913881ffd0a21aec456d',
  'c6fffebce06eaee367e3741ad1a8acb8b3b618c1dab2b5d4f25307c2fb2625',
  '5bf3cac6b93a2ac8d714dccd3793d6bd484f7b65fe1bf9cf6692ae3965a507e',
  '987246a8e5faba5c0eea2a501b5a2a657b3024563420e36b554dfff2d624a5',
  '157df4a4253cef5dd5f722f29629677ddd77121de8ce58da6f3352ec1c3de

In [26]:
newDf = pd.DataFrame(cluster, columns=['name', 'url', 'ram', 'rom', 'price', 'createdDate', 'color', 'status', 'url_img','store', 'id_orgin', 'list_id'])

In [28]:
newDf.to_json('./search.json', 'records')